# वरदान


Team: CRACK_ILLUSION (Swayam Jathar & Erum Noor)


There are approximately 66 million people in India suffering from either complete or partial deafness, and at least 50lakh are children. These people are specially abled and not able to communicate with other beings, easily. To communicate with others they use sign language, which becomes tough for others to understand it. So circumscribing the this major issue we decided to simplfy the mode of communication between specially abled people and normal people. Introducing "वरदान" which works as a trasnlator from Sign Language to Text. The motto of "वरदान" is o create a meaningful connection with specially abled people.

Instaltalling the prerequisite librabries. Further libraries will install when we need it in the following code.

In [8]:
from torch.utils.data import Dataset
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.nn as nn
import numpy as np
import torch

from typing import List

import csv


class SignLanguageMNIST(Dataset):

    @staticmethod
    def get_label_mapping():
  
        mapping = list(range(25))
        mapping.pop(9)
        return mapping

    @staticmethod
    def read_label_samples_from_csv(path: str):
        
        mapping = SignLanguageMNIST.get_label_mapping()
        labels, samples = [], []
        with open(path) as f:
            _ = next(f)  # skip header
            for line in csv.reader(f):
                label = int(line[0])
                labels.append(mapping.index(label))
                samples.append(list(map(int, line[1:])))
        return labels, samples

    def __init__(self,
            path: str="sign_mnist_train.csv",
            mean: List[float]=[0.485],
            std: List[float]=[0.229]):
      
        labels, samples = SignLanguageMNIST.read_label_samples_from_csv(path)
        self._samples = np.array(samples, dtype=np.uint8).reshape((-1, 28, 28, 1))
        self._labels = np.array(labels, dtype=np.uint8).reshape((-1, 1))

        self._mean = mean
        self._std = std

    def __len__(self):
        return len(self._labels)

    def __getitem__(self, idx):
        transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.RandomResizedCrop(28, scale=(0.8, 1.2)),
            transforms.ToTensor(),
            transforms.Normalize(mean=self._mean, std=self._std)])

        return {
            'image': transform(self._samples[idx]).float(),
            'label': torch.from_numpy(self._labels[idx]).float()
        }


def get_train_test_loaders(batch_size=32):
    trainset = SignLanguageMNIST('sign_mnist_train.csv')
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

    testset = SignLanguageMNIST('sign_mnist_test.csv')
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)
    return trainloader, testloader


if __name__ == '__main__':
    loader, _ = get_train_test_loaders(2)
    print(next(iter(loader)))

{'image': tensor([[[[0.2624, 0.2967, 0.3138,  ..., 0.7248, 0.7077, 0.7077],
          [0.2796, 0.3138, 0.3309,  ..., 0.7419, 0.7248, 0.7248],
          [0.2967, 0.3138, 0.3481,  ..., 0.7762, 0.7591, 0.7419],
          ...,
          [0.5022, 0.5364, 0.6049,  ..., 1.1872, 1.1872, 1.1872],
          [0.4851, 0.5193, 0.6049,  ..., 1.2043, 1.1872, 1.1872],
          [0.4679, 0.5193, 0.6049,  ..., 1.2043, 1.1872, 1.1872]]],


        [[[0.9646, 1.0159, 1.0844,  ..., 0.9988, 0.9817, 0.9303],
          [0.9817, 1.0331, 1.0844,  ..., 1.0331, 0.9988, 0.9646],
          [0.9988, 1.0502, 1.1015,  ..., 1.0502, 1.0331, 0.9817],
          ...,
          [1.4440, 1.5125, 1.5468,  ..., 1.4954, 1.4612, 1.4269],
          [1.4440, 1.4954, 1.5297,  ..., 1.4954, 1.4783, 1.4440],
          [1.3927, 1.4440, 1.5810,  ..., 1.4783, 1.4612, 1.4098]]]]), 'label': tensor([[16.],
        [19.]])}


We have used a technique called data augmentation, where samples are disturbed during training, to increase the model’s robustness to these perturbations. In particular, randomly zoom in on the image by varying amounts and on different locations, via RandomResizedCrop. To normalize the inputs so that image values are rescaled to the [0, 1] range in expectation, instead of [0, 255]; to accomplish this, we used the dataset _mean and _std when normalizing.

Dataset is being taken from Kaggle: https://www.kaggle.com/datamunge/sign-language-mnist


In [9]:
from torch.utils.data import Dataset
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch




class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 6, 3)
        self.conv3 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 48)
        self.fc3 = nn.Linear(48, 24)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def main():
    net = Net().float()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    trainloader, _ = get_train_test_loaders()
    for epoch in range(12):  # loop over the dataset multiple times
        train(net, criterion, optimizer, trainloader, epoch)
        scheduler.step()
    torch.save(net.state_dict(), "checkpoint.pth")


def train(net, criterion, optimizer, trainloader, epoch):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs = Variable(data['image'].float())
        labels = Variable(data['label'].long())
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels[:, 0])
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 0:
            print('[%d, %5d] loss: %.6f' % (epoch, i, running_loss / (i + 1)))


if __name__ == '__main__':
    main()

[0,     0] loss: 3.178262
[0,   100] loss: 3.177437
[0,   200] loss: 3.175903
[0,   300] loss: 3.170432
[0,   400] loss: 3.076334
[0,   500] loss: 2.852937
[0,   600] loss: 2.605645
[0,   700] loss: 2.378769
[0,   800] loss: 2.176646
[1,     0] loss: 0.429329
[1,   100] loss: 0.530480
[1,   200] loss: 0.480089
[1,   300] loss: 0.446717
[1,   400] loss: 0.410137
[1,   500] loss: 0.387869
[1,   600] loss: 0.365638
[1,   700] loss: 0.338897
[1,   800] loss: 0.324282
[2,     0] loss: 0.270173
[2,   100] loss: 0.186629
[2,   200] loss: 0.180923
[2,   300] loss: 0.160046
[2,   400] loss: 0.157416
[2,   500] loss: 0.152737
[2,   600] loss: 0.150285
[2,   700] loss: 0.142306
[2,   800] loss: 0.137463
[3,     0] loss: 0.066076
[3,   100] loss: 0.113861
[3,   200] loss: 0.112179
[3,   300] loss: 0.110203
[3,   400] loss: 0.105925
[3,   500] loss: 0.102603
[3,   600] loss: 0.096586
[3,   700] loss: 0.094647
[3,   800] loss: 0.092531
[4,     0] loss: 0.018499
[4,   100] loss: 0.056363
[4,   200] l

We see a neral network train as output of the above code. Hence we have deployed a deep neural network.

In [10]:
from torch.utils.data import Dataset
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import numpy as np

import onnx
import onnxruntime as ort




def evaluate(outputs: Variable, labels: Variable) -> float:
    """Evaluate neural network outputs against non-one-hotted labels."""
    Y = labels.numpy()
    Yhat = np.argmax(outputs, axis=1)
    return float(np.sum(Yhat == Y))


def batch_evaluate(
        net: Net,
        dataloader: torch.utils.data.DataLoader) -> float:
    """Evaluate neural network in batches, if dataset is too large."""
    score = n = 0.0
    for batch in dataloader:
        n += len(batch['image'])
        outputs = net(batch['image'])
        if isinstance(outputs, torch.Tensor):
            outputs = outputs.detach().numpy()
        score += evaluate(outputs, batch['label'][:, 0])
    return score / n


def validate():
    trainloader, testloader = get_train_test_loaders()
    net = Net().float().eval()

    pretrained_model = torch.load("checkpoint.pth")
    net.load_state_dict(pretrained_model)

    print('=' * 10, 'PyTorch', '=' * 10)
    train_acc = batch_evaluate(net, trainloader) * 100.
    print('Training accuracy: %.1f' % train_acc)
    test_acc = batch_evaluate(net, testloader) * 100.
    print('Validation accuracy: %.1f' % test_acc)

    trainloader, testloader = get_train_test_loaders(1)

    # export to onnx
    fname = "signlanguage.onnx"
    dummy = torch.randn(1, 1, 28, 28)
    torch.onnx.export(net, dummy, fname, input_names=['input'])

    # check exported model
    model = onnx.load(fname)
    onnx.checker.check_model(model)  # check model is well-formed

    # create runnable session with exported model
    ort_session = ort.InferenceSession(fname)
    net = lambda inp: ort_session.run(None, {'input': inp.data.numpy()})[0]

    print('=' * 10, 'ONNX', '=' * 10)
    train_acc = batch_evaluate(net, trainloader) * 100.
    print('Training accuracy: %.1f' % train_acc)
    test_acc = batch_evaluate(net, testloader) * 100.
    print('Validation accuracy: %.1f' % test_acc)


if __name__ == '__main__':
    validate()

========== PyTorch ==========
Training accuracy: 99.8
Validation accuracy: 97.5
========== ONNX ==========
Training accuracy: 99.8
Validation accuracy: 97.4


We obtain following accuracy for training and validation dataset. 


In [11]:
import cv2
import numpy as np
import onnxruntime as ort


def center_crop(frame):
    h, w, _ = frame.shape
    start = abs(h - w) // 2
    if h > w:
        return frame[start: start + w]
    return frame[:, start: start + h]


def main():
    # constants
    index_to_letter = list('ABCDEFGHIKLMNOPQRSTUVWXY')
    mean = 0.485 * 255.
    std = 0.229 * 255.

    # create runnable session with exported model
    ort_session = ort.InferenceSession("signlanguage.onnx")

    cap = cv2.VideoCapture(0)
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        # preprocess data
        frame = center_crop(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        x = cv2.resize(frame, (28, 28))
        x = (x - mean) / std

        x = x.reshape(1, 1, 28, 28).astype(np.float32)
        y = ort_session.run(None, {'input': x})[0]

        index = np.argmax(y, axis=1)
        letter = index_to_letter[int(index)]

        cv2.putText(frame, letter, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 255, 0), thickness=2)
        cv2.imshow("Sign Language Translator", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()